In [92]:
import torch
import numpy as np
import pandas as pd
import requests
import json
from torch import nn
from torch.nn.utils import rnn
from torch.utils.data import RandomSampler, BatchSampler

## Definining Model Architecture
Training on LSTM to try and capture some hidden state of a team

In [451]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, dropout=0.2, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear1 = nn.Linear(hidden_size, 64)
        self.bn = nn.BatchNorm1d(619)

        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.35)
        self.linear2 = nn.Linear(64, 2)
        
        #affine, bn, relu, do

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        lstm_out, _ = self.lstm(x)
#         print(lstm_out.shape)
        dropout = self.dropout(lstm_out)
        linear1 = self.linear1(dropout)
        normd = self.bn(linear1)
        relud = self.relu(normd)
#         print(relud.shape)
 
        dropped = self.dropout2(relu)
        pred = self.linear2(dropped)
        
        return pred

In [452]:
raw_df = pd.read_csv('2019pbpfeatures_calvin.csv')
raw_df.head(2)

,Date,URL,Location,Time,WinningTeam,Quarter,SecLeft,AwayPlay,HomePlay,AwayTeam,...,HomeAssistTotal,AwayDRBTotal,HomeDRBTotal,AwayORBTotal,HomeORBTotal,AwayFoulTotal,HomeFoulTotal,HomeFinalScore,AwayFinalScore,HomeWin
0,2019-10-22 00:00:00,/boxscores/201910220LAC.html,STAPLES Center Los Angeles California,2021-05-06 22:30:00,LAC,1,720,Jump ball: J. McGee vs. I. Zubac (L. James gai...,NaN,LAL,...,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,0.8,0.133333,1
1,2019-10-22 00:00:00,/boxscores/201910220LAC.html,STAPLES Center Los Angeles California,2021-05-06 22:30:00,LAC,1,707,L. James makes 2-pt layup from 3 ft (assist by...,NaN,LAL,...,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,0.8,0.133333,1


### Testing for Single Team
Picking the Warriors ('GSW') for now

In [453]:
# df = raw_df[(raw_df['HomeTeam'] == 'GSW') | (raw_df['AwayTeam'] == 'GSW')].reset_index()
# df.drop(['index'],1,inplace=True)


In [454]:
# raw_df.head()

In [455]:
len(raw_df)

539265

In [456]:
##train tests split
df = raw_df
sizes = df.groupby(['Date']).size()
mask = np.random.rand(len(sizes)) < 0.8
final_mask = np.array([])
for i in range(len(sizes)):
    final_mask = np.concatenate((final_mask, np.full(sizes[i], mask[i])))
final_mask = final_mask.astype(bool)
df_test = df[~final_mask]

df_train_val = df[final_mask]

sizes2 = df_train_val.groupby(['Date']).size()
mask2 = np.random.rand(len(sizes)) < 0.75
final_mask2 = np.array([])
for i in range(len(sizes2)):
    final_mask2 = np.concatenate((final_mask2, np.full(sizes2[i], mask2[i])))
final_mask2 = final_mask2.astype(bool)

df_train = df_train_val[final_mask2]
df_val = df_train_val[~final_mask2]

In [457]:
len(df_train), len(df_train.groupby('URL'))

(310263, 657)

In [458]:
len(df_train), len(df_val), len(df_test)

(310263, 147433, 81569)

In [459]:
df_train.head()

,Date,URL,Location,Time,WinningTeam,Quarter,SecLeft,AwayPlay,HomePlay,AwayTeam,...,HomeAssistTotal,AwayDRBTotal,HomeDRBTotal,AwayORBTotal,HomeORBTotal,AwayFoulTotal,HomeFoulTotal,HomeFinalScore,AwayFinalScore,HomeWin
0,2019-10-22 00:00:00,/boxscores/201910220LAC.html,STAPLES Center Los Angeles California,2021-05-06 22:30:00,LAC,1,720,Jump ball: J. McGee vs. I. Zubac (L. James gai...,NaN,LAL,...,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,0.8,0.133333,1
1,2019-10-22 00:00:00,/boxscores/201910220LAC.html,STAPLES Center Los Angeles California,2021-05-06 22:30:00,LAC,1,707,L. James makes 2-pt layup from 3 ft (assist by...,NaN,LAL,...,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,0.8,0.133333,1
2,2019-10-22 00:00:00,/boxscores/201910220LAC.html,STAPLES Center Los Angeles California,2021-05-06 22:30:00,LAC,1,700,NaN,L. Shamet misses 3-pt jump shot from 27 ft,LAL,...,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,0.8,0.133333,1
3,2019-10-22 00:00:00,/boxscores/201910220LAC.html,STAPLES Center Los Angeles California,2021-05-06 22:30:00,LAC,1,698,Defensive rebound by A. Davis,NaN,LAL,...,-5.081362,-6.173513,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,0.8,0.133333,1
4,2019-10-22 00:00:00,/boxscores/201910220LAC.html,STAPLES Center Los Angeles California,2021-05-06 22:30:00,LAC,1,683,D. Green makes 3-pt jump shot from 25 ft (assi...,NaN,LAL,...,-5.081362,-6.173513,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,0.8,0.133333,1


In [460]:
df_train.columns

Index(['Date', 'URL', 'Location', 'Time', 'WinningTeam', 'Quarter', 'SecLeft',
       'AwayPlay', 'HomePlay', 'AwayTeam', 'AwayScore', 'HomeTeam',
       'HomeScore', 'ShotType', 'ShotOutcome', 'FoulType', 'Fouler', 'Fouled',
       'FreeThrowShooter', 'FreeThrowOutcome', 'FreeThrowNum',
       'TurnoverPlayer', 'TurnoverType', 'TurnoverCause', 'TurnoverCauser',
       'Assister', 'ReboundType', 'Rebounder', 'hour', 'hour_sin', 'hour_cos',
       'AwayPlayOneHot', 'HomePlayOneHot', 'AwayAssist', 'HomeAssist',
       '2-pt shot', '3-pt shot', 'Home 2-pt shot', 'Away 2-pt shot',
       'Home 3-pt shot', 'Away 3-pt shot', 'HomeMade 2-pt shot',
       'AwayMade 2-pt shot', 'HomeMade 3-pt shot', 'AwayMade 3-pt shot',
       'AwayDRB', 'HomeDRB', 'AwayORB', 'HomeORB', 'AwayFoul', 'HomeFoul',
       'AwayTurnover', 'HomeTurnover', 'AwayFreeThrow', 'HomeFreeThrow',
       'AwayMadeFreeThrow', 'HomeMadeFreeThrow', 'GameTimeLeft',
       'GameTimeLeftSin', 'GameTimeLeftCos', 'Home 2-pt shotTotal

In [461]:
homeLabels = df_train.groupby('URL')['HomeFinalScore'].max()
awayLabels = df_train.groupby('URL')['AwayFinalScore'].max()
df_train.drop(['Location', 'Time', 'WinningTeam', 'Quarter', 'SecLeft',
       'AwayPlay', 'HomePlay', 'AwayScore', 'HomeScore', 'ShotType',
       'ShotOutcome', 'FoulType', 'Fouler', 'Fouled', 'FreeThrowShooter',
       'FreeThrowOutcome', 'FreeThrowNum', 'TurnoverPlayer', 'TurnoverType',
       'TurnoverCause', 'TurnoverCauser', 'Assister', 'ReboundType', 'Rebounder', 'hour', 'hour_sin', 'hour_cos',
       'AwayPlayOneHot', 'HomePlayOneHot', '2-pt shot', '3-pt shot', 'GameTimeLeft', 'HomeWin', 'HomeFinalScore', 'AwayFinalScore',
        ],1,inplace=True)

homeLabelsVal = df_val.groupby('URL')['HomeFinalScore'].max()
awayLabelsVal = df_val.groupby('URL')['AwayFinalScore'].max()
df_val.drop(['Location', 'Time', 'WinningTeam', 'Quarter', 'SecLeft',
       'AwayPlay', 'HomePlay', 'AwayScore', 'HomeScore', 'ShotType',
       'ShotOutcome', 'FoulType', 'Fouler', 'Fouled', 'FreeThrowShooter',
       'FreeThrowOutcome', 'FreeThrowNum', 'TurnoverPlayer', 'TurnoverType',
       'TurnoverCause', 'TurnoverCauser', 'Assister', 'ReboundType', 'Rebounder', 'hour', 'hour_sin', 'hour_cos',
       'AwayPlayOneHot', 'HomePlayOneHot', '2-pt shot', '3-pt shot', 'GameTimeLeft', 'HomeWin', 'HomeFinalScore', 'AwayFinalScore',
        ],1,inplace=True)
df_train.head()

,Date,URL,AwayTeam,HomeTeam,AwayAssist,HomeAssist,Home 2-pt shot,Away 2-pt shot,Home 3-pt shot,Away 3-pt shot,...,AwayMadeFreeThrowTotal,HomeMadeFreeThrowTotal,AwayAssistTotal,HomeAssistTotal,AwayDRBTotal,HomeDRBTotal,AwayORBTotal,HomeORBTotal,AwayFoulTotal,HomeFoulTotal
0,2019-10-22 00:00:00,/boxscores/201910220LAC.html,LAL,LAC,0,0,0,0,0,0,...,-2.94465,-2.847606,-4.925531,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027
1,2019-10-22 00:00:00,/boxscores/201910220LAC.html,LAL,LAC,1,0,0,1,0,0,...,-2.94465,-2.847606,-4.718451,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027
2,2019-10-22 00:00:00,/boxscores/201910220LAC.html,LAL,LAC,0,0,0,0,1,0,...,-2.94465,-2.847606,-4.718451,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027
3,2019-10-22 00:00:00,/boxscores/201910220LAC.html,LAL,LAC,0,0,0,0,0,0,...,-2.94465,-2.847606,-4.718451,-5.081362,-6.173513,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027
4,2019-10-22 00:00:00,/boxscores/201910220LAC.html,LAL,LAC,1,0,0,0,0,1,...,-2.94465,-2.847606,-4.511371,-5.081362,-6.173513,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027


In [462]:
df_train.columns, len(df_train), len(df_train.columns)

(Index(['Date', 'URL', 'AwayTeam', 'HomeTeam', 'AwayAssist', 'HomeAssist',
        'Home 2-pt shot', 'Away 2-pt shot', 'Home 3-pt shot', 'Away 3-pt shot',
        'HomeMade 2-pt shot', 'AwayMade 2-pt shot', 'HomeMade 3-pt shot',
        'AwayMade 3-pt shot', 'AwayDRB', 'HomeDRB', 'AwayORB', 'HomeORB',
        'AwayFoul', 'HomeFoul', 'AwayTurnover', 'HomeTurnover', 'AwayFreeThrow',
        'HomeFreeThrow', 'AwayMadeFreeThrow', 'HomeMadeFreeThrow',
        'GameTimeLeftSin', 'GameTimeLeftCos', 'Home 2-pt shotTotal',
        'Away 2-pt shotTotal', 'HomeMade 2-pt shotTotal',
        'AwayMade 2-pt shotTotal', 'Home 3-pt shotTotal', 'Away 3-pt shotTotal',
        'HomeMade 3-pt shotTotal', 'AwayMade 3-pt shotTotal',
        'AwayTurnoverTotal', 'HomeTurnoverTotal', 'AwayFreeThrowTotal',
        'HomeFreeThrowTotal', 'AwayMadeFreeThrowTotal',
        'HomeMadeFreeThrowTotal', 'AwayAssistTotal', 'HomeAssistTotal',
        'AwayDRBTotal', 'HomeDRBTotal', 'AwayORBTotal', 'HomeORBTotal',
       

In [463]:
df_train.groupby('URL').size().max()

605

In [464]:
len(df_train.groupby('URL')), len(df_val.groupby('URL')), 

(657, 312)

In [465]:
game_urls = df_train['URL'].unique()
game_urls_val = df_val['URL'].unique()
sampler = BatchSampler(RandomSampler(game_urls), 4, drop_last=False)
game_urls[:5]

array(['/boxscores/201910220LAC.html', '/boxscores/201910220TOR.html',
       '/boxscores/201910230BRK.html', '/boxscores/201910230CHO.html',
       '/boxscores/201910230DAL.html'], dtype=object)

In [466]:
# for indices in sampler:
#     print(indices)

In [467]:
cols = ['GameTimeLeftSin', 'GameTimeLeftCos', 'Home 2-pt shot', 'Away 2-pt shot',
        'Home 3-pt shot', 'Away 3-pt shot', 'HomeMade 2-pt shot',
        'AwayMade 2-pt shot', 'HomeMade 3-pt shot', 'AwayMade 3-pt shot',
        'AwayDRB', 'HomeDRB', 'AwayORB', 'HomeORB',
        'AwayFoul', 'HomeFoul',
        'AwayTurnover', 'HomeTurnover', 'AwayFreeThrow', 'HomeFreeThrow',
        'AwayMadeFreeThrow', 'HomeMadeFreeThrow',
        'Home 2-pt shotTotal', 'Away 2-pt shotTotal', 'HomeMade 2-pt shotTotal',
        'AwayMade 2-pt shotTotal', 'Home 3-pt shotTotal', 'Away 3-pt shotTotal',
        'HomeMade 3-pt shotTotal', 'AwayMade 3-pt shotTotal',
        'AwayTurnoverTotal', 'HomeTurnoverTotal', 'AwayFreeThrowTotal',
        'HomeFreeThrowTotal', 'AwayMadeFreeThrowTotal',
        'HomeMadeFreeThrowTotal', 'AwayAssistTotal', 'HomeAssistTotal',
        'AwayDRBTotal', 'HomeDRBTotal', 'AwayORBTotal', 'HomeORBTotal',
        'AwayFoulTotal', 'HomeFoulTotal']

In [468]:
df_train['listified'] = df_train[cols].apply(list, axis=1).aggregate(list)
df_val['listified'] = df_val[cols].apply(list, axis=1).aggregate(list)

df_train.head()

/Users/calvingrewal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/calvingrewal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Date,URL,AwayTeam,HomeTeam,AwayAssist,HomeAssist,Home 2-pt shot,Away 2-pt shot,Home 3-pt shot,Away 3-pt shot,...,HomeMadeFreeThrowTotal,AwayAssistTotal,HomeAssistTotal,AwayDRBTotal,HomeDRBTotal,AwayORBTotal,HomeORBTotal,AwayFoulTotal,HomeFoulTotal,listified
0,2019-10-22 00:00:00,/boxscores/201910220LAC.html,LAL,LAC,0,0,0,0,0,0,...,-2.847606,-4.925531,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,"[-2.4492935982947064e-16, 1.0, 0.0, 0.0, 0.0, ..."
1,2019-10-22 00:00:00,/boxscores/201910220LAC.html,LAL,LAC,1,0,0,1,0,0,...,-2.847606,-4.718451,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,"[-0.028357798245379123, 0.999597836771706, 0.0..."
2,2019-10-22 00:00:00,/boxscores/201910220LAC.html,LAL,LAC,0,0,0,0,1,0,...,-2.847606,-4.718451,-5.081362,-6.347203,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,"[-0.043619387365335306, 0.9990482215818578, 0...."
3,2019-10-22 00:00:00,/boxscores/201910220LAC.html,LAL,LAC,0,0,0,0,0,0,...,-2.847606,-4.718451,-5.081362,-6.173513,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,"[-0.047978128521343866, 0.9988483864849508, 0...."
4,2019-10-22 00:00:00,/boxscores/201910220LAC.html,LAL,LAC,1,0,0,0,0,1,...,-2.847606,-4.511371,-5.081362,-6.173513,-6.707985,-3.386662,-3.530054,-4.566096,-4.487027,"[-0.08063384351427176, 0.996743790188891, 0.0,..."


In [469]:
train_data = df_train.groupby('URL')['listified'].aggregate(list)
train_data = train_data.tolist()

In [470]:
val_data = df_val.groupby('URL')['listified'].aggregate(list)
val_data = val_data.tolist()

In [471]:
len(train_data),len(train_data[0][0]), train_data[0], 

(657,
 44,
 [[-2.4492935982947064e-16,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -6.540084370390012,
   -6.596201027211362,
   -5.320539090307989,
   -5.335444870441754,
   -4.8361179420976566,
   -4.853960209634527,
   -3.1995916618626032,
   -3.2363539286603986,
   -3.5512782926657405,
   -3.529433115908964,
   -3.18121621277943,
   -3.117166561055918,
   -2.9446496121628107,
   -2.847605962886945,
   -4.925530658445746,
   -5.0813618368831595,
   -6.347202858741713,
   -6.707984629102177,
   -3.3866623339782245,
   -3.5300541791779656,
   -4.566096417636128,
   -4.4870267800594315],
  [-0.028357798245379123,
   0.999597836771706,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -6.540084370390012,
   -6.474311806063475,
   -5.320539090307989,
 

In [472]:
len(homeLabels), len(homeLabelsVal)

(657, 312)

In [473]:
a = [[-1000] * 5]
a * 2

[[-1000, -1000, -1000, -1000, -1000], [-1000, -1000, -1000, -1000, -1000]]

In [474]:
def pad_data(data):
    pad_length = 619
    vec_length = len(data[0][0])
    blank_pad = [[-1000] * vec_length]
    masks = [] # index of the last unpadded input for each game
    for game in data:
    #     print(len(game), len(game[0]))
        padding = pad_length - len(game)
        masks.append(int(0.75 * (len(game)-1)))
        game.extend(blank_pad * padding)

In [475]:
pad_data(train_data)
pad_data(val_data)

In [476]:
train_data = torch.tensor(train_data)
train_data.shape

torch.Size([657, 619, 44])

In [477]:
val_data = torch.tensor(val_data)
val_data.shape

torch.Size([312, 619, 44])

In [478]:
train_data[0,masks[0], :]

tensor([ 0.9974,  0.0719,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.7327, -1.5987,
        -0.6908, -1.5451, -1.5753, -1.7525, -0.8664, -0.5854, -1.0917, -1.0939,
        -0.2601, -1.1252, -0.4478, -1.2814, -1.6123, -1.1831, -2.1786, -2.0586,
        -0.6095, -1.3513, -0.4685, -0.2942])

In [479]:
masks[0]

342

In [480]:
train_data[0,masks[0]+1, :]

tensor([ 0.9952,  0.0980,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.7327, -1.5987,
        -0.6908, -1.5451, -1.5753, -1.6115, -0.8664, -0.5854, -1.0917, -1.0939,
        -0.2601, -1.1252, -0.4478, -1.2814, -1.6123, -1.1831, -2.1786, -2.0586,
        -0.6095, -1.3513, -0.4685, -0.2942])

In [481]:
masks = np.array(masks)
def build_batch(data, homeLabels, awayLabels, indices):
    batch_data = data[indices]
    batch_labels_home = torch.tensor(homeLabels[indices].tolist())
    batch_labels_away = torch.tensor(awayLabels[indices].tolist())
#     print('hi')
    batch_masks = masks[indices]
    for i in range(batch_data.shape[0]):
        batch_data[i, batch_masks[i]+1:] = -1000
    return batch_data, batch_labels_home, batch_labels_away, batch_masks

In [482]:
data, _, _, _ = build_batch(train_data, homeLabels, awayLabels, [0])
data[0, masks[0]]

tensor([ 0.9974,  0.0719,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.7327, -1.5987,
        -0.6908, -1.5451, -1.5753, -1.7525, -0.8664, -0.5854, -1.0917, -1.0939,
        -0.2601, -1.1252, -0.4478, -1.2814, -1.6123, -1.1831, -2.1786, -2.0586,
        -0.6095, -1.3513, -0.4685, -0.2942])

In [483]:
# in_size = 32
batch_size = 4
time_steps = 619

num_epochs = 5
lr = 4e-3
model = LSTM_Model(train_data.shape[-1], 512)
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [237]:
# with torch.no_grad():
#     for indices in BatchSampler(RandomSampler(game_urls), batch_size, drop_last=False):
#         batch_data, batch_labels_home, batch_labels_away, batch_masks = build_batch(indices)
# #             print(type(batch_data), type(batch_labels_home), type(batch_labels_away))
#         output_seq = model(batch_data)
#         print(output_seq.shape)

# #         last_output = output_seq[:, -1,:]
#             B = last_output.shape[0]
#             last_output = torch.zeros(B, 2) # (B, 2)
#     #         test = torch.zeros(B, 2)
#             for i in range(B):
#                 last_output[i] = output_seq[i, batch_masks[i], :]            
#         print(last_output.shape, output_seq)
# #       print(batch_labels_home.shape, batch_labels_away.shape,)
#         target = torch.hstack(
#                     (batch_labels_home.reshape(-1, 1), batch_labels_away.reshape(-1, 1))
#                 )
#         loss = criterion(last_output, target)
#         print(loss.item())
# #             print(torch.hstack((batch_labels_home.reshape(-1, 1), batch_labels_away.reshape(-1, 1))))

In [ ]:
train_losses = {}
val_losses = {}

In [485]:
model.train()
print(f'training with batch_size: {batch_size} for {num_epochs} epochs with lr: {lr}')
for epoch in range(num_epochs):
    for i, indices in enumerate(BatchSampler(RandomSampler(game_urls), batch_size, drop_last=False)):
        batch_data, batch_labels_home, batch_labels_away, batch_masks = build_batch(train_data, homeLabels, awayLabels, indices)
        # batch_data has shape(B, 619, 19), the 619 is the padded sequence length, and the 19 is the feature vector length for each timesetep
        output_seq = model(batch_data)
        
#         last_output = output_seq[:,-1,:]
        B = output_seq.shape[0]
        last_output = torch.zeros(B, 2) # sinnce we have a many-to-one problem, we only want the last element in each sequence
        for j in range(B): # spent way too long tryna figure out the cool way to do this, but i guess a for loop works
            last_output[j] = output_seq[j, batch_masks[j], :]   
#         print(last_output.shape)
        
        target = torch.hstack(
            (batch_labels_home.reshape(-1, 1), batch_labels_away.reshape(-1, 1))
        )
        loss = criterion(last_output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f'epoch: {epoch}, i: {i}', loss.item())
            print('actual: ', target * 15 + 100)
            print('predicted: ', last_output * 15 + 100)
            train_losses[(epoch, i)] = loss.item()
        if i % 100 == 0:
            with torch.no_grad():
                val_indices = list(range(len(game_urls_val)))
                batch_data, batch_labels_home, batch_labels_away, batch_masks = build_batch(val_data, homeLabelsVal, awayLabelsVal, val_indices)
                output_seq = model(batch_data)
                B = output_seq.shape[0]
                last_output = torch.zeros(B, 2) 
                for j in range(B):
                    last_output[j] = output_seq[j, batch_masks[j], :]   
                target = torch.hstack(
                    (batch_labels_home.reshape(-1, 1), batch_labels_away.reshape(-1, 1))
                )
                loss = criterion(last_output, target)
                val_losses[(epoch, i)] = loss.item()
                print(f'************ epoch: {epoch} i: {i} val loss: {loss.item()} **************')
#                 print('actual: ', target * 15 + 100)
#                 print('predicted: ', last_output * 15 + 100)
            
        
# for i in range(len(df_train)):
#     # Forward pass: Compute predicted y by passing x to the model
#     x = torch.FloatTensor(df_train.iloc[i]).unsqueeze(0).unsqueeze(0)
#     y = torch.FloatTensor((homeLabels.iloc[i], awayLabels.iloc[i])).unsqueeze(0).unsqueeze(0)
#     y_pred = model(x)

# #     # Compute and print loss
#     loss = criterion(y_pred, y)
#     if i % 100 == 99:
#         print(i, loss.item())

# #     # Zero gradients, perform a backward pass, and update the weights.
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

training with batch_size: 4 for 5 epochs with lr: 0.004
epoch: 0, i: 0 0.4715120494365692
actual:  tensor([[101., 115.],
        [ 97., 112.],
        [ 98., 102.],
        [109., 110.]])
predicted:  tensor([[108.6134,  98.4493],
        [110.3824, 105.6212],
        [112.6115, 103.8137],
        [117.1934, 106.3753]], grad_fn=<AddBackward0>)
************ epoch: 0 i: 0 val loss: 3.6392016410827637 **************
epoch: 0, i: 10 0.6197474598884583
actual:  tensor([[100.,  99.],
        [107., 119.],
        [111., 112.],
        [ 98., 100.]])
predicted:  tensor([[107.5460, 111.8615],
        [117.2894, 120.4205],
        [112.7698, 118.6540],
        [116.3275, 120.0497]], grad_fn=<AddBackward0>)
epoch: 0, i: 20 1.1253716945648193
actual:  tensor([[126., 125.],
        [ 91., 111.],
        [118., 108.],
        [ 94.,  93.]])
predicted:  tensor([[117.4085, 113.4597],
        [124.1686, 102.7319],
        [110.9711, 105.7116],
        [109.5915, 111.7729]], grad_fn=<AddBackward0>)
epoc

************ epoch: 1 i: 100 val loss: 0.6802781820297241 **************
epoch: 1, i: 110 0.6799824237823486
actual:  tensor([[ 87., 108.],
        [113.,  97.],
        [124., 115.],
        [101., 115.]])
predicted:  tensor([[116.5588, 113.2703],
        [111.2536, 106.9210],
        [116.3309, 111.1795],
        [110.9884, 108.0849]], grad_fn=<AddBackward0>)
epoch: 1, i: 120 0.3822563886642456
actual:  tensor([[120., 101.],
        [115., 105.],
        [128., 123.],
        [ 99.,  94.]])
predicted:  tensor([[109.4749, 107.2410],
        [108.6528, 105.4510],
        [117.6392, 112.9339],
        [111.1481, 105.8993]], grad_fn=<AddBackward0>)
epoch: 1, i: 130 1.51485276222229
actual:  tensor([[105.,  85.],
        [ 95.,  80.],
        [ 88., 127.],
        [108., 117.]])
predicted:  tensor([[115.3577, 111.8997],
        [109.6362, 105.9219],
        [113.1311, 111.0558],
        [118.4414, 120.8576]], grad_fn=<AddBackward0>)
epoch: 1, i: 140 1.1271954774856567
actual:  tensor([[13

epoch: 3, i: 50 0.2815866470336914
actual:  tensor([[103.,  99.],
        [105., 106.],
        [ 93., 100.],
        [117., 113.]])
predicted:  tensor([[109.3576, 103.4935],
        [113.2915, 111.5496],
        [110.0859, 104.9278],
        [119.2936, 118.0227]], grad_fn=<AddBackward0>)
epoch: 3, i: 60 0.5923334360122681
actual:  tensor([[118.,  86.],
        [112., 125.],
        [102., 104.],
        [123., 105.]])
predicted:  tensor([[113.9810, 109.6948],
        [114.2715, 110.4610],
        [112.0024, 104.8826],
        [112.6924, 113.0616]], grad_fn=<AddBackward0>)
epoch: 3, i: 70 0.296366810798645
actual:  tensor([[103., 114.],
        [123., 112.],
        [109.,  94.],
        [122., 118.]])
predicted:  tensor([[112.3860, 106.2465],
        [111.4464, 109.0365],
        [112.0323, 106.3624],
        [117.6891, 110.1025]], grad_fn=<AddBackward0>)
epoch: 3, i: 80 0.2585618495941162
actual:  tensor([[101., 113.],
        [123., 108.],
        [120., 125.],
        [ 97., 108.]]

KeyboardInterrupt: 

In [442]:
cols = ['GameTimeLeftSin', 'GameTimeLeftCos', 'Home 2-pt shot', 'Away 2-pt shot',
        'Home 3-pt shot', 'Away 3-pt shot', 'HomeMade 2-pt shot',
        'AwayMade 2-pt shot', 'HomeMade 3-pt shot', 'AwayMade 3-pt shot',
        'AwayDRB', 'HomeDRB', 'AwayORB', 'HomeORB',
        'AwayFoul', 'HomeFoul',
        'AwayTurnover', 'HomeTurnover', 'AwayFreeThrow', 'HomeFreeThrow',
        'AwayMadeFreeThrow', 'HomeMadeFreeThrow',
        'Home 2-pt shotTotal', 'Away 2-pt shotTotal', 'HomeMade 2-pt shotTotal',
        'AwayMade 2-pt shotTotal', 'Home 3-pt shotTotal', 'Away 3-pt shotTotal',
        'HomeMade 3-pt shotTotal', 'AwayMade 3-pt shotTotal',
        'AwayTurnoverTotal', 'HomeTurnoverTotal', 'AwayFreeThrowTotal',
        'HomeFreeThrowTotal', 'AwayMadeFreeThrowTotal',
        'HomeMadeFreeThrowTotal', 'AwayAssistTotal', 'HomeAssistTotal',
        'AwayDRBTotal', 'HomeDRBTotal', 'AwayORBTotal', 'HomeORBTotal',
        'AwayFoulTotal', 'HomeFoulTotal']

In [443]:
test_game_urls = df_test['URL'].unique()

In [444]:
# testHomeLabels = df_test['HomeFinalScore']
# testAwayLabels = df_test['AwayFinalScore']

testHomeLabels = df_test.groupby('URL')['HomeFinalScore'].max()
testAwayLabels = df_test.groupby('URL')['AwayFinalScore'].max()

df_test['listified'] = df_test[cols].apply(list, axis=1).aggregate(list)

test_data = df_test.groupby('URL')['listified'].aggregate(list)
test_data = test_data.tolist()

# df_test.drop(['AwayTeam', 'HomeTeam','Date','HomeFinalScore','AwayFinalScore'],1,inplace=True)

/Users/calvingrewal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [445]:
pad_data(test_data)
test_data = torch.tensor(test_data)
test_data.shape

torch.Size([174, 619, 44])

In [446]:
# start_test = df_test.loc[df['GameTimeLeft'] == 1]
# testHomeLabels = start_test['HomeFinalScore']
# testAwayLabels = start_test['AwayFinalScore']
# testHomeWins = (start_test['AwayFinalScore'] > start_test['HomeFinalScore']).astype(int)
# start_test.drop(['AwayTeam', 'HomeTeam','Date','HomeFinalScore','AwayFinalScore'],1,inplace=True)

In [447]:
# len(testHomeWins)

In [448]:
# model.eval()
# preds = []
# for i in range(len(start_test)):
#     x = torch.FloatTensor(start_test.iloc[i]).unsqueeze(0).unsqueeze(0)
#     y = torch.FloatTensor((testHomeLabels.iloc[i], testAwayLabels.iloc[i]))
    
#     pred = model(x)
    
#     if (pred[0,0,0].item() > pred[0,0,1].item()):
#         preds.append(True)
#     else:
#         preds.append(False)
# preds

    

In [449]:
model.eval()
indices = list(range(len(test_game_urls)))

batch_data, batch_labels_home, batch_labels_away, batch_masks = build_batch(test_data,testHomeLabels, testAwayLabels, indices)

output_seq = model(batch_data)
B = output_seq.shape[0]
last_output = torch.zeros(B, 2)
for j in range(B): 
    last_output[j] = output_seq[j, batch_masks[j]-1, :]   

target = torch.hstack(
    (batch_labels_home.reshape(-1, 1), batch_labels_away.reshape(-1, 1))
)
loss = criterion(last_output, target)

In [450]:
print(f"test loss: {loss}")

test loss: 0.3749949634075165


In [411]:
last_output[:4] * 15 + 100

tensor([[114.0648, 112.1210],
        [107.3943, 108.8500],
        [105.2431, 108.2555],
        [113.9297, 118.6250]], grad_fn=<AddBackward0>)

In [412]:
target[:4] * 15 + 100

tensor([[112., 102.],
        [130., 122.],
        [100., 117.],
        [122., 141.]])

In [367]:
batch_masks

array([342, 360, 393, 359, 386, 380, 405, 340, 374, 357, 351, 346, 372,
       368, 341, 377, 381, 450, 393, 391, 351, 313, 375, 351, 418, 356,
       348, 404, 327, 384, 344, 358, 390, 364, 371, 366, 373, 351, 336,
       385, 378, 358, 372, 352, 366, 378, 322, 384, 324, 323, 320, 380,
       348, 369, 338, 354, 373, 352, 387, 347, 372, 352, 332, 357, 338,
       347, 360, 364, 348, 347, 362, 351, 365, 375, 348, 357, 351, 370,
       354, 387, 397, 363, 355, 381, 363, 320, 363, 369, 422, 368, 340,
       362, 314, 369, 339, 339, 333, 438, 381, 350, 378, 411, 351, 370,
       382, 342, 312, 391, 349, 423, 369, 338, 292, 328, 343, 418, 359,
       354, 345, 363, 375, 337, 379, 375, 339, 355, 339, 340, 348, 372,
       363, 330, 362, 337, 370, 367, 336, 313, 345, 407, 369, 375, 399,
       398, 344, 348, 399, 285, 349, 341, 351, 342, 348, 331, 348, 332,
       326, 360, 368, 341, 381, 351, 372, 339, 335, 375, 399, 354, 375,
       339, 348, 397, 342, 363, 344, 349, 334, 375, 354, 348, 34